# LLMs for malware

Easily prompt and label LLM prompts.

In [4]:
import logging
import time
import os
import sys
import ipywidgets as widgets
import csv

from ipywidgets import interact, interact_manual
from IPython.display import Markdown, clear_output
from langchain_community.chat_models import ChatDeepInfra
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Make sure you use your own API token
# Simply place the key in the deep_infra.key file
with open("./deep_infra.key", 'r') as f:
    os.environ["DEEPINFRA_API_TOKEN"] = f.read().strip()
user = "ciprian"

if not os.path.exists("logs"):
    os.makedirs("logs")

logger = logging.getLogger(__name__)
logging.basicConfig(filename=f"./logs/{user}_output_{time.time()}.log", encoding='utf-8', level=logging.DEBUG)

"Initialized successfully"

'Initialized successfully'

In [6]:
@interact_manual.options(manual_name="Run")(
        system_prompt = widgets.Textarea(value="Be a helpful assistant", description="System prompt:", layout=widgets.Layout(height="64px", width="auto")),
        prompt = widgets.Textarea(description="Prompt:", layout=widgets.Layout(height="64px", width="auto")),
        taxonomy = widgets.Text(description="Taxonomy pos.:"),
        language = widgets.Text(description="Language:"),
        terms = widgets.ToggleButtons(options=["Academic", "Script-kiddie"], description="Terms used:"),
        # label = widgets.Text(description="Label:"),
        detail = widgets.ToggleButtons(options=["Low", "Medium", "High"], description="Level of detail:"))
def call_model(system_prompt, prompt, taxonomy, language, terms, detail):
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=prompt
        )
    ]

    display(Markdown("_Running..._"))

    chat = ChatDeepInfra(model="google/gemma-1.1-7b-it",
                         temperature=0.7,
                         max_tokens=4096)

    res = chat.invoke(messages)

    clear_output()
    display(Markdown(res.content))

    # save = widgets.Button(description="Save")
    # def on_save(b):
    #     print("click")
    # save.on_click(on_save)

    # display(save)
    @interact_manual.options(manual_name="Save")()
    def on_save():

        with open("prompts.csv", "a") as f:
            writer = csv.writer(f)
            writer.writerow((prompt, taxonomy, language, terms, detail))

        print("Saved!")
        
    logger.debug(dict(res))

interactive(children=(Textarea(value='Be a helpful assistant', description='System prompt:', layout=Layout(hei…